# Objectif du script

Ce script vise à trouver des correspondances entre deux bases de données de festivals :

- **`df_top_50`** : contient les 50 festivals les plus populaires, avec des informations sur leur nom et lieu.
- **`df_france`** : contient des informations détaillées sur des festivals en France, y compris des coordonnées géographiques.

## Objectif

L'objectif est de trouver le meilleur match entre les festivals des deux bases en combinant :

- La similarité des **noms de festivals**.
- La similarité des **communes de déroulement**.
- La **proximité géographique** des coordonnées.

Une fois les correspondances identifiées, les données sont fusionnées pour enrichir les informations de `df_top_50` avec celles de `df_france`.

Les festivals sans correspondance sont également identifiés et sauvegardés dans un fichier CSV pour une analyse ultérieure.


In [9]:
!pip install fuzzywuzzy
!pip install geopy
!pip install folium
!pip install python-Levenshtein

# Importer les bibliothèques

In [10]:
import numpy as np
import pandas as pd
import folium
from geopy.distance import geodesic
from fuzzywuzzy import process, fuzz


# Charger les données

In [ ]:
# Charger les fichiers CSV
df_top_50 = pd.read_csv("50_festivals_geolocalises.csv")
df_france = pd.read_csv("festivals_en_France.csv", sep=";", encoding="utf-8")

# Afficher un aperçu des deux DataFrames
print("Aperçu des 50 festivals les plus populaires :")
print(df_top_50.head())

print("\nAperçu des festivals en France :")
print(df_france.head())


FileNotFoundError: [Errno 2] No such file or directory: 'festivals_en France.csv'

In [ ]:
# Normaliser les noms pour la comparaison (en minuscule, suppression des espaces autour)
df_top_50["Nom_clean"] = df_top_50["Nom"].str.lower().str.strip()
df_top_50["Commune_clean"] = df_top_50["Lieu"].str.split("\n").str[0].str.lower().str.strip()

df_france["Nom_clean"] = df_france["Nom du festival"].str.lower().str.strip()
df_france["Commune_clean"] = df_france["Commune principale de déroulement"].str.lower().str.strip()


# Nettoyer les noms pour la correspondance

In [1]:
# Afficher les noms des colonnes
print(df_france.columns.tolist())

NameError: name 'df_france' is not defined

In [18]:
# Nettoyer les noms de colonnes pour supprimer les caractères invisibles
df_france.columns = df_france.columns.str.strip().str.replace('\ufeff', '')

# Changement de la colonne correspondant aux coordonnées 
df_france[['Latitude', 'Longitude']] = df_france['Géocodage xy'].str.split(',', expand=True).astype(float)

# Afficher les colonnes après nettoyage
print(df_france.columns.tolist())

# Afficher un aperçu
print("Festivals populaires :")
print(df_top_50.head())
print("\nFestivals en France :")
print(df_france.head())


['Nom du festival', 'Envergure territoriale', 'Région principale de déroulement', 'Département principal de déroulement', 'Commune principale de déroulement', 'Code postal (de la commune principale de déroulement)', 'Code Insee commune', 'Code Insee EPCI', 'Libellé EPCI', 'Numéro de voie', 'Type de voie (rue, Avenue, boulevard, etc.)', 'Nom de la voie', 'Adresse postale', "Complément d'adresse (facultatif)", 'Site internet du festival', 'Adresse e-mail', 'Décennie de création du festival', 'Année de création du festival', 'Discipline dominante', 'Sous-catégorie spectacle vivant', 'Sous-catégorie musique', 'Sous-catégorie Musique CNM', 'Sous-catégorie cinéma et audiovisuel', 'Sous-catégorie arts visuels et arts numériques', 'Sous-catégorie livre et littérature', 'Période principale de déroulement du festival', 'Identifiant Agence A', 'Identifiant', 'Géocodage xy', 'identifiant CNM', 'Latitude', 'Longitude']
Festivals populaires :
   Classement                     Nom                    

In [27]:
def find_best_match(row, df_reference, threshold_distance=10, score_threshold=70):
    """
    Trouve le meilleur match dans df_reference en combinant :
    - Distance géographique
    - Similitude des noms
    - Similitude des communes
    """
    best_match = None
    best_score = -np.inf

    for _, ref_row in df_reference.iterrows():
        # Calculer la similarité des noms
        score_name = fuzz.token_sort_ratio(row["Nom_clean"], ref_row["Nom_clean"])
        score_commune = fuzz.token_sort_ratio(row["Commune_clean"], ref_row["Commune_clean"])

        # Calculer la distance géographique si les coordonnées existent
        if "Latitude" in row and "Longitude" in row and "Latitude" in ref_row and "Longitude" in ref_row:
            try:
                coords1 = (row["Latitude"], row["Longitude"])
                coords2 = (ref_row["Latitude"], ref_row["Longitude"])
                distance = geodesic(coords1, coords2).km
            except:
                distance = np.inf
        else:
            distance = np.inf

        # Combiner les scores
        combined_score = score_name + score_commune
        if distance < threshold_distance:
            combined_score += 20  # Bonus si distance faible

        # Mettre à jour le meilleur match
        if combined_score > best_score and score_name >= score_threshold:
            best_match = ref_row["Nom du festival"]
            best_score = combined_score

    return best_match


In [28]:
# Appliquer la fonction de correspondance
print("Recherche des correspondances...")
df_top_50["Festival match"] = df_top_50.apply(
    lambda row: find_best_match(row, df_france), axis=1
)

# Afficher les résultats
print("\nCorrespondances trouvées :")
print(df_top_50[["Nom", "Lieu", "Festival match"]])


Recherche des correspondances...

Correspondances trouvées :
                               Nom                      Lieu  \
0            FESTIVAL RETRO C TROP                  Tilloloy   
1            PRINTEMPS DE PEROUGES   Saint Maurice de Remens   
2              PAROLES ET MUSIQUES             Saint Etienne   
3             MUSICALARUE FESTIVAL                     Luxey   
4           FESTIVAL PAUSE GUITARE                      Albi   
5      FRANCOFOLIES DE LA ROCHELLE               La Rochelle   
6                 LUXEXPO OPEN AIR                Luxembourg   
7                 GUITARE EN SCENE  Saint Julien En Genevois   
8                         HELLFEST                   Clisson   
9            ECAUSSYSTEME FESTIVAL                    Gignac   
10  FESTIVAL LES VIEILLES CHARRUES          Carhaix Plouguer   
11                BACCHUS FESTIVAL           Argeles Sur Mer   
12   LES DEFERLANTES SUD DE FRANCE               Le Barcares   
13         BOBITAL, L'ARMOR A SONS         

In [29]:
# Festivals sans correspondance
festivals_sans_match = df_top_50[df_top_50["Festival match"].isna()]
print("\nFestivals sans correspondance :")
print(festivals_sans_match[["Nom", "Lieu"]])

# Sauvegarder les festivals sans match dans un fichier CSV
festivals_sans_match.to_csv("festivals_sans_match.csv", index=False, encoding="utf-8")
print("Les festivals sans match ont été sauvegardés dans 'festivals_sans_match.csv'.")



Festivals sans correspondance :
                              Nom         Lieu
6                LUXEXPO OPEN AIR   Luxembourg
12  LES DEFERLANTES SUD DE FRANCE  Le Barcares
14                      XMAS VYBE     Bordeaux
24                    BOOMIN FEST       Nantes
Les festivals sans match ont été sauvegardés dans 'festivals_sans_match.csv'.


# Fonction pour réussir à fusionner les deux bases de données en cherchant des similitudes entre les données 

In [19]:
def find_best_match(row, df_reference, threshold_distance=5.0, score_threshold=80):
    """
    Trouve le meilleur match dans df_reference en combinant les critères :
    - Distance géographique
    - Similitude des noms
    - Similitude des communes
    """
    best_match = None
    best_score = -np.inf

    # Comparer chaque festival dans df_reference
    for _, ref_row in df_reference.iterrows():
        score = 0

        # 1. Correspondance géographique
        if pd.notna(row["Latitude"]) and pd.notna(row["Longitude"]) \
           and pd.notna(ref_row["Latitude"]) and pd.notna(ref_row["Longitude"]):
            distance = geodesic((row["Latitude"], row["Longitude"]), 
                                (ref_row["Latitude"], ref_row["Longitude"])).km
            if distance <= threshold_distance:
                score += 50  # Ajouter des points si les coordonnées sont proches

        # 2. Correspondance par nom du festival
        if pd.notna(row["Nom"]) and pd.notna(ref_row["Nom du festival"]):
            name_similarity = fuzz.token_sort_ratio(row["Nom"], ref_row["Nom du festival"])
            if name_similarity >= score_threshold:
                score += name_similarity  # Ajouter des points selon la similarité du nom

        # 3. Correspondance par commune
        if pd.notna(row["Lieu"]) and pd.notna(ref_row["Commune principale de déroulement"]):
            commune_similarity = fuzz.token_sort_ratio(row["Lieu"], ref_row["Commune principale de déroulement"])
            if commune_similarity >= score_threshold:
                score += commune_similarity  # Ajouter des points selon la similarité de la commune

        # Mettre à jour le meilleur match
        if score > best_score:
            best_score = score
            best_match = ref_row["Nom du festival"]

    return best_match


In [20]:
# Appliquer la fonction de correspondance à chaque festival populaire
df_top_50["Festival match"] = df_top_50.apply(
    lambda row: find_best_match(row, df_france), axis=1
)

# Afficher les correspondances trouvées
print("\nCorrespondances trouvées :")
print(df_top_50[["Nom", "Lieu", "Festival match"]].head())



Correspondances trouvées :
                      Nom                     Lieu  \
0   FESTIVAL RETRO C TROP                 Tilloloy   
1   PRINTEMPS DE PEROUGES  Saint Maurice de Remens   
2     PAROLES ET MUSIQUES            Saint Etienne   
3    MUSICALARUE FESTIVAL                    Luxey   
4  FESTIVAL PAUSE GUITARE                     Albi   

                  Festival match  
0                   Rétro C trop  
1           Printemps de Paroles  
2            Paroles et musiques  
3                    Musicalarue  
4  Festival Pause guitare à Albi  


# On fusionne les bases de données à partir des similitudes trouvées 

In [21]:
# Fusionner les deux DataFrames sur le match des festivals
df_merged = pd.merge(
    df_top_50,
    df_france,
    left_on="Festival match",
    right_on="Nom du festival",
    how="left",
    suffixes=("_top50", "_france")
)

# Afficher un aperçu des données fusionnées
print("\nDonnées fusionnées :")
print(df_merged.head())

# Sauvegarder les données fusionnées dans un CSV
df_merged.to_csv("festivals_fusionnes_complets.csv", index=False, encoding="utf-8")
print("Les données ont été sauvegardées dans 'festivals_fusionnes_complets.csv'.")



Données fusionnées :
   Classement                     Nom                     Lieu  \
0           1   FESTIVAL RETRO C TROP                 Tilloloy   
1           4   PRINTEMPS DE PEROUGES  Saint Maurice de Remens   
2           6     PAROLES ET MUSIQUES            Saint Etienne   
3           8    MUSICALARUE FESTIVAL                    Luxey   
4          10  FESTIVAL PAUSE GUITARE                     Albi   

                                                Lien  Latitude_top50  \
0  https://infoconcert.com/festival/festival-retr...       49.644546   
1  https://infoconcert.com/festival/printemps-de-...       45.958549   
2  https://infoconcert.com/festival/paroles-et-mu...       45.440147   
3  https://infoconcert.com/festival/musicalarue-f...       44.263963   
4  https://infoconcert.com/festival/festival-paus...       43.927755   

   Longitude_top50                 Festival match  \
0         2.749010                   Rétro C trop   
1         5.278103           Printemps de 

# On cherche les raisons pour lequelles certains festivals n'ont pas trouvé de match dans la base qui recense tous les festivals en France

In [25]:
# Filtrer les festivals où 'Festival match' est NaN ou vide
festivals_sans_match = df_top_50[df_top_50["Festival match"].isna() | (df_top_50["Festival match"] == "")]

# Afficher les festivals sans correspondance
print("Festivals sans correspondance :")
print(festivals_sans_match[["Nom", "Lieu"]])

# Sauvegarder ces festivals dans un CSV pour analyse ultérieure
festivals_sans_match.to_csv("festivals_sans_match.csv", index=False, encoding="utf-8")
print("Les festivals sans match ont été sauvegardés dans 'festivals_sans_match.csv'.")


Festivals sans correspondance :
Empty DataFrame
Columns: [Nom, Lieu]
Index: []
Les festivals sans match ont été sauvegardés dans 'festivals_sans_match.csv'.


In [24]:
# Vérifier les valeurs uniques dans Festival match
print("Valeurs uniques dans 'Festival match' :")
print(df_top_50["Festival match"].unique())


Valeurs uniques dans 'Festival match' :
['Rétro C trop' 'Printemps de Paroles' 'Paroles et musiques' 'Musicalarue'
 'Festival Pause guitare à Albi' 'Fema - Festival La Rochelle Cinéma'
 'Festival andalou' 'Guitare en scène' 'Hellfest Open Air'
 'Or notes festival' 'Les Vieilles Charrues' 'Maghreb si loin, si proche'
 "Bobital - L'Armor à Sons"
 "Rencontres Cinématographiques La Classe Ouvrière C'est Pas du Cinéma"
 'Les soirées Sacha Guitry' 'Garorock' 'Ardèche Aluna festival'
 'Festival du Bout du Monde' "Festival d'Humour et de Jazz"
 'Festival Cognac Blues Passions' "La Nuit De l'Erdre" 'Marsatac'
 'Europavox' 'Nature Nomade' 'Salon des littératures maudites'
 "Festival Flam'" 'Surgères Brass Band Festival'
 'Fête du Bruit dans Landerneau' 'Festival russe']


In [23]:
# Filtrer les festivals sans correspondance
festivals_sans_match = df_top_50[df_top_50["Festival match"].isna()]

# Sauvegarder les festivals sans correspondance dans un CSV
festivals_sans_match.to_csv("festivals_sans_match.csv", index=False, encoding="utf-8")
print("Festivals sans correspondance sauvegardés dans 'festivals_sans_match.csv'.")

# Afficher un aperçu
print("Festivals sans correspondance :")
print(festivals_sans_match[["Nom", "Lieu"]])


Festivals sans correspondance sauvegardés dans 'festivals_sans_match.csv'.
Festivals sans correspondance :
Empty DataFrame
Columns: [Nom, Lieu]
Index: []
